In [1]:
import pandas as pd
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

On récupère les bases de données nettoyées:

In [2]:
data_2015 = pd.read_csv('data_2015_cleaned.csv')
data_2016 = pd.read_csv('data_2016_cleaned.csv')

# Régressions avec les données déclaratives
On  va à présent réaliser des régressions pour prédire les variables suivantes, **sans utiliser le score ENERGYSTAR**:
- TotalGHGEmissions
- SiteEnergyUse(kBtu)

## Définitions générales
On défini les entrées pour nos régressions ainsi que la sortie attendue.

In [3]:
# cols_in: Colonnes en entrée dans les régressions
cols_in = ['YearBuilt', 'NumberofFloors', 'PropertyGFABuilding(s)', 'Latitude', 'Longitude']
# col_out: Colonne en sortie dans les régressions
col_out = 'SiteEnergyUse(kBtu)'

Comme on l'a vu durant l'ACP, **la variable avec le nombre de bâtiments n'est pas petinente**. On l'exclu donc de nos entrées.

In [4]:
data = pd.concat([data_2015, data_2016])

In [5]:
reg_in = data[cols_in]  # Tableau 2d avec les valeurs des variables pour chaque entrée
reg_out = data[col_out]  # Liste de vecteurs à 1d avec les valeurs de sortie
# Standardisation des entrées
# std_scale: Scaler pour nos données d'entrée
std_scale = preprocessing.StandardScaler().fit(reg_in)
reg_in_scale = std_scale.transform(reg_in)  # Nos entrées standardisées

## Régressions pour SiteEnergyUse(kBtu)

### Baseline - Régression linéaire classique

In [6]:
def fit_with_cv(regression, reg_name, scoring='r2', cv=3):
    """Entraîne un modèle avec validation croisée et affiche les temps d'exécution"""
    # scores: Scores obtenus avec la validation croisée
    scores = cross_validate(regression, reg_in_scale, reg_out, scoring='r2', cv=3)
    print("Score avec %s: R2=%.3f" % (reg_name, scores['test_score'].mean()))
    print("Temps entraînement: %.3fs" % scores['fit_time'].sum())
    print("Temps scoring: %.3fs" % scores['score_time'].sum())

In [7]:
# reg_lin: Régression linéaire
reg_lin = linear_model.LinearRegression()
fit_with_cv(reg_lin, "régression linéaire")

Score avec régression linéaire: R2=0.485
Temps entraînement: 0.042s
Temps scoring: 0.005s


On va maintenant essayer plusieurs autres types de régressions pour améliorer les performances de cette régression de base.

### Ridge
On va réaliser une régression Rigde, avec une **recherche en grille** des meilleurs paramètres et une **validation croisée**.

In [8]:
# ridge: Régression Ridge
rigde = linear_model.Ridge(alpha=25)
fit_with_cv(rigde, "régression Rigde")

Score avec régression Rigde: R2=0.485
Temps entraînement: 0.006s
Temps scoring: 0.002s


Avec cette régression, augmenter le alpha ne fait que diminuer le R2: **Limiter la taille des poids n'a pas d'impact.** Cela signifie qu'il n'y à pas de sur-apprentissage, ni de corrélations entre nos entrées. On s'attend à observer le même phénomène pour la régression Lasso.

### Lasso
On va maintenant appliquer une régression Lasso.

In [9]:
# lasso: Régressions Lasso
lasso = linear_model.Lasso(alpha=1000)
fit_with_cv(lasso, "régression Lasso")

Score avec régression Lasso: R2=0.485
Temps entraînement: 0.004s
Temps scoring: 0.001s


Quelque soit la forme de la norme utilisée, **limiter le poids des features n'a pas d'impact**

À plus forte raison, **utiliser l'elastic net n'a pas d'intérêt** ici.

### k-nn
Essayon d'appliquer la forme régressive de l'algorithme des plus proches voisins.

In [10]:
knn = KNeighborsRegressor(n_neighbors=2)
fit_with_cv(knn, "k-nn")

Score avec k-nn: R2=0.723
Temps entraînement: 0.008s
Temps scoring: 0.037s


Quand on augmente le nombre de voisins pris en compte, on diminue le R2. **Ce n'est donc probablement pas le bon type de modèle**, même si il est déjà bien meilleur que les régressions linéaires.

### Forêt aléatoire 

In [11]:
# rand_forest: Régression de forêt aléatoire
rand_forest = RandomForestRegressor(n_estimators=13, max_depth=14)
fit_with_cv(rand_forest, "forêt aléatoire")

Score avec forêt aléatoire: R2=0.799
Temps entraînement: 0.395s
Temps scoring: 0.013s


Par rapport au modèle précédent, **on a gagné ~8 points**.

### Gradient boosting

In [12]:
# grad_boost: Régression X-Gradient boosting optimisée avec recherche par grille
grad_boost = GradientBoostingRegressor(learning_rate=0.4, n_estimators=50, max_depth=10)
fit_with_cv(grad_boost, "gradient boosting")

Score avec gradient boosting: R2=0.851
Temps entraînement: 1.975s
Temps scoring: 0.013s


On optimise ce modèle qui a donné les meilleurs résultats.

In [13]:
params = {'learning_rate': [0.35+i*0.02 for i in range(6)],
          'n_estimators':[48+i for i in range(5)],
          'max_depth': [8+i for i in range(5)]}
# grid_grad_boost: Régression X-Gradient boosting optimisée avec recherche par grille
grid_grad_boost = GridSearchCV(GradientBoostingRegressor(), params, cv=3)
grid_grad_boost.fit(reg_in_scale, reg_out)
cv_results = grid_grad_boost.cv_results_
print("Score Gradient boosting optimisé: %.2f" % grid_grad_boost.best_score_)
print("Temps entraînement: %.3fs" % (cv_results['mean_fit_time'].sum()+grid_grad_boost.refit_time_))
print("Temps score: %.3fs" % cv_results['mean_score_time'].sum())

Score Gradient boosting optimisé: 0.87
Temps entraînement: 94.439s
Temps score: 0.646s


### Conclusion
Notre modèle final utilisant **le gradient bossting atteint un R2 de 0.87**, sachant qu'avec une régression linéaire simple, on atteignait à peine 0.54.

## Régressions pour TotalGHGEmissions

In [14]:
col_out = 'TotalGHGEmissions'

In [15]:
reg_out = data[col_out]  # Liste de vecteurs à 1d avec les valeurs de sortie

In [16]:
# reg_lin: Régression linéaire
reg_lin = linear_model.LinearRegression()
fit_with_cv(reg_lin, "régression linéaire")

Score avec régression linéaire: R2=0.282
Temps entraînement: 0.009s
Temps scoring: 0.003s


On va maintenant essayer plusieurs autres types de régressions pour améliorer les performances de cette régression de base.

### Ridge
On va réaliser une régression Rigde, avec une **recherche en grille** des meilleurs paramètres et une **validation croisée**.

In [17]:
# ridge: Régression Ridge
rigde = linear_model.Ridge(alpha=25)
fit_with_cv(rigde, "régression Rigde")

Score avec régression Rigde: R2=0.282
Temps entraînement: 0.004s
Temps scoring: 0.001s


Avec cette régression, augmenter le alpha ne fait que diminuer le R2: **Limiter la taille des poids n'a pas d'impact.**

### Lasso
On va maintenant appliquer une régression Lasso.

In [18]:
# lasso: Régressions Lasso
lasso = linear_model.Lasso(alpha=1)
fit_with_cv(lasso, "régression Lasso")

Score avec régression Lasso: R2=0.282
Temps entraînement: 0.003s
Temps scoring: 0.001s


Quelque soit la forme de la norme utilisée, **limiter le poids des features n'a pas d'impact**

À plus forte raison, **utiliser l'elastic net n'a pas d'intérêt** ici.

### k-nn
Essayon d'appliquer la forme régressive de l'algorithme des plus proches voisins.

In [19]:
knn = KNeighborsRegressor(n_neighbors=2)
fit_with_cv(knn, "k-nn")

Score avec k-nn: R2=0.572
Temps entraînement: 0.010s
Temps scoring: 0.040s


Quand on augmente le nombre de voisins pris en compte, on diminue le R2. **Ce n'est donc probablement pas le bon type de modèle**, même si il est déjà bien meilleur que les régressions linéaires.

### Forêt aléatoire 

In [20]:
# rand_forest: Régression de forêt aléatoire
rand_forest = RandomForestRegressor(n_estimators=13, max_depth=14)
fit_with_cv(rand_forest, "forêt aléatoire")

Score avec forêt aléatoire: R2=0.713
Temps entraînement: 0.390s
Temps scoring: 0.013s


Par rapport au modèle précédent, **on gagne 24 points**, ce qui est considérable.

### Gradient boosting

In [21]:
# grad_boost: Régression X-Gradient boosting optimisée avec recherche par grille
grad_boost = GradientBoostingRegressor(learning_rate=0.4, n_estimators=50, max_depth=10)
fit_with_cv(grad_boost, "gradient boosting")

Score avec gradient boosting: R2=0.789
Temps entraînement: 1.854s
Temps scoring: 0.013s


On optimise ce modèle qui a donné les meilleurs résultats.

In [22]:
params = {'learning_rate': [0.35+i*0.02 for i in range(6)],
          'n_estimators':[48+i for i in range(5)],
          'max_depth': [8+i for i in range(5)]}
# grid_grad_boost: Régression X-Gradient boosting optimisée avec recherche par grille
grid_grad_boost = GridSearchCV(GradientBoostingRegressor(), params, cv=3)
grid_grad_boost.fit(reg_in_scale, reg_out)
cv_results = grid_grad_boost.cv_results_
print("Score Gradient boosting optimisé: %.2f" % grid_grad_boost.best_score_)
print("Temps entraînement: %.3fs" % (cv_results['mean_fit_time'].sum()+grid_grad_boost.refit_time_))
print("Temps score: %.3fs" % cv_results['mean_score_time'].sum())

Score Gradient boosting optimisé: 0.81
Temps entraînement: 93.833s
Temps score: 0.629s


### Conclusion
Notre modèle final utilisant **le gradient bossting atteint un R2 de 0.81**, sachant qu'avec une régression linéaire simple, on atteignait seulement 0.31.

# Régressions avec l'ENERGYSTARScore
En plus des données déclaratives, ajoutons l'ENERGYSTARScore pour évaluer son impact sur le modèle.

In [23]:
data = data.dropna(subset=['ENERGYSTARScore'])
cols_in.append('ENERGYSTARScore')

## Régressions pour SiteEnergyUse(kBtu)

On re-définit la sortie...

In [24]:
col_out = 'SiteEnergyUse(kBtu)'

... Ainsi que nos données d'entraînement et de test.

In [25]:
reg_in = data[cols_in]  # Tableau 2d avec les valeurs des variables pour chaque entrée
reg_out = data[col_out]  # Liste de vecteurs à 1d avec les valeurs de sortie
# std_scale: Scaler pour nos données d'entrée
std_scale = preprocessing.StandardScaler().fit(reg_in)
reg_in_scale = std_scale.transform(reg_in)  # Nos entrées standardisées

Voyons l'**impact sur notre modèle final**:

In [26]:
params = {'learning_rate': [0.35+i*0.02 for i in range(6)],
          'n_estimators':[48+i for i in range(5)],
          'max_depth': [8+i for i in range(5)]}
# grid_grad_boost: Régression X-Gradient boosting optimisée avec recherche par grille
grid_grad_boost = GridSearchCV(GradientBoostingRegressor(), params, cv=3)
grid_grad_boost.fit(reg_in_scale, reg_out)
cv_results = grid_grad_boost.cv_results_
print("Score Gradient boosting optimisé: %.2f" % grid_grad_boost.best_score_)
print("Temps entraînement: %.3fs" % (cv_results['mean_fit_time'].sum()+grid_grad_boost.refit_time_))
print("Temps score: %.3fs" % cv_results['mean_score_time'].sum())

Score Gradient boosting optimisé: 0.86
Temps entraînement: 80.022s
Temps score: 0.562s


On constate que **cet ajout diminue le R2 d'un point**. 

## Régressions pour TotalGHGEmissions

On re-définit la sortie...

In [6]:
col_out = 'TotalGHGEmissions'

... Ainsi que nos données d'entraînement et de test.

In [7]:
reg_in = data[cols_in]  # Tableau 2d avec les valeurs des variables pour chaque entrée
reg_out = data[col_out]  # Liste de vecteurs à 1d avec les valeurs de sortie
# Standardisation des entrées
# std_scale: Scaler pour nos données d'entrée
std_scale = preprocessing.StandardScaler().fit(reg_in)
reg_in_scale = std_scale.transform(reg_in)  # Nos entrées standardisées

Voyons l'**impact sur notre modèle final**:

In [8]:
params = {'learning_rate': [0.35+i*0.02 for i in range(6)],
          'n_estimators':[48+i for i in range(5)],
          'max_depth': [8+i for i in range(5)]}
# grid_grad_boost: Régression X-Gradient boosting optimisée avec recherche par grille
grid_grad_boost = GridSearchCV(GradientBoostingRegressor(), params, cv=3)
grid_grad_boost.fit(reg_in_scale, reg_out)
cv_results = grid_grad_boost.cv_results_
print("Score Gradient boosting optimisé: %.2f" % grid_grad_boost.best_score_)
print("Temps entraînement: %.3fs" % (cv_results['mean_fit_time'].sum()+grid_grad_boost.refit_time_))
print("Temps score: %.3fs" % cv_results['mean_score_time'].sum())

Score Gradient boosting optimisé: 0.81
Temps entraînement: 92.874s
Temps score: 0.615s


Avec le score energy star, **Le R2 ne change pas.**

### Conclusion
L'ajout de l'Energy Star Score n'a pas d'intérêt pour nos modèles.